In [2]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

## Initialization

### Imports

In [3]:
import os
import sys
import cv2
import json
import torch
import zipfile
import numpy as np
import pandas as pd
import plotly.express as px

from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader

sys.path.append("../code/")

In [11]:
from params import *

from training.main import validate
from model_zoo.models import define_model
from utils.torch import load_model_weights

### Load

In [5]:
df_info = pd.read_csv(DATA_PATH + f"HuBMAP-20-dataset_information.csv")
df_mask = pd.read_csv(DATA_PATH + "train.csv")
df = pd.read_csv(OUT_PATH + "df_images.csv")

In [7]:
def k_fold_inf(config, df, log_folder=None, use_full_size=True):
    """
    Performs a patient grouped k-fold cross validation.
    The following things are saved to the log folder : val predictions, val indices, histories

    Args:
        config (Config): Parameters.
        df (pandas dataframe): Metadata.
        log_folder (None or str, optional): Folder to logs results to. Defaults to None.
    """
    folds = df[config.cv_column].unique()
    cvs = []

    for i, fold in enumerate(folds):
        if i in config.selected_folds:
            print(f"\n-------------   Fold {i + 1} / {len(folds)}  -------------\n")
            df_val = df[df[config.cv_column] == fold].reset_index()
            
            val_images = df_val["tile_name"].apply(lambda x: x.split("_")[0]).unique()

            model = define_model(
                config.decoder,
                config.encoder,
                num_classes=config.num_classes,
                encoder_weights=config.encoder_weights,
            ).to(config.device)
            model.zero_grad()
            
            load_model_weights(model, log_folder + f"{config.decoder}_{config.encoder}_{i}.pt")
            
            scores, thresholds = validate(
                model,
                config,
                val_images,
                log_folder=log_folder,
                use_full_size=use_full_size,
            )

In [8]:
# log_folder = "../logs/2020-12-21/3/"  # se_resnext50
# log_folder = "../logs/2020-12-22/9/"  # b4
log_folder = "../logs/2020-12-22/10/"  # b5 
# log_folder = "../logs/2020-12-23/2/"  # b6

In [9]:
class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

config = json.load(open(log_folder + 'config.json', 'r'))
config = Config(**config)

In [12]:
metrics = k_fold_inf(config, df, log_folder=log_folder, use_full_size=False)


-------------   Fold 1 / 4  -------------


 -> Loading weights from ../logs/2020-12-23/2/Unet_efficientnet-b6_0.pt


    -> Validating 

 - Scored 0.9233 for image 2f6ecfcdf with threshold 0.35
 - Scored 0.8907 for image aaa6a05cc with threshold 0.40
